### Libs

In [25]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
# from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

### Data loading

In [78]:
df = pd.read_csv('df_for_EDA.csv') 
#                  names = ['Kline open time','Open price','High price','Low price','Close price',
#                             'Volume','Kline Close time','Quote asset volume','Number of trades',
#                             'Taker buy base asset volume','Taker buy quote asset volume','Unused field, ignore'])

In [79]:
df

,Kline open time,time,Open price,High price,Low price,Close price,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,1672531200000,2023-01-01 00:00:00,72.38,72.39,72.36,72.39,782.0,56602.48,17,560.0,40536.27
1,1672532100000,2023-01-01 00:15:00,72.39,72.39,72.37,72.37,3706.0,268254.08,22,2484.0,179809.76
2,1672533000000,2023-01-01 00:30:00,72.39,72.39,72.38,72.39,1197.0,86649.00,19,1014.0,73403.46
3,1672533900000,2023-01-01 00:45:00,72.39,72.39,72.38,72.38,818.0,59212.19,12,535.0,38728.65
4,1672534800000,2023-01-01 01:00:00,72.38,72.39,72.38,72.38,182.0,13173.59,11,43.0,3112.77
...,...,...,...,...,...,...,...,...,...,...,...
10708,1682172000000,2023-04-22 14:00:00,83.68,83.69,83.58,83.61,16091.0,1345937.69,133,3926.0,328472.59
10709,1682172900000,2023-04-22 14:15:00,83.61,83.66,83.59,83.59,8023.0,670952.67,103,4106.0,343434.97
10710,1682173800000,2023-04-22 14:30:00,83.61,83.63,83.56,83.63,9958.0,832463.27,106,5336.0,446140.40
10711,1682174700000,2023-04-22 14:45:00,83.63,83.63,83.56,83.61,10409.0,870135.37,144,5220.0,436410.33


In [80]:
df['time'] = pd.to_datetime(df['time'])

In [81]:
df['diff_time'] = df.time.diff()

In [82]:
df[df.diff_time == df.diff_time.max()]

,Kline open time,time,Open price,High price,Low price,Close price,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,diff_time
7923,1679666400000,2023-03-24 14:00:00,77.69,77.8,77.69,77.77,17045.0,1325647.4,157,11805.0,918196.09,0 days 01:30:00


Пробел в 1.5 часа довольно большой - из объявлений твитера Бинанса видим, что 24 марта проводилось Техническое обслуживание системы спотовой торговли Binance.

In [83]:
df.loc[7917:7923]

,Kline open time,time,Open price,High price,Low price,Close price,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,diff_time
7917,1679656500000,2023-03-24 11:15:00,77.66,77.70,77.64,77.68,3390.0,263312.62,74,2516.0,195437.26,0 days 00:15:00
7918,1679657400000,2023-03-24 11:30:00,77.68,77.68,77.68,77.68,0.0,0.00,0,0.0,0.00,0 days 00:15:00
7919,1679658300000,2023-03-24 11:45:00,77.68,77.68,77.68,77.68,0.0,0.00,0,0.0,0.00,0 days 00:15:00
7920,1679659200000,2023-03-24 12:00:00,77.68,77.68,77.68,77.68,0.0,0.00,0,0.0,0.00,0 days 00:15:00
7921,1679660100000,2023-03-24 12:15:00,77.68,77.68,77.68,77.68,0.0,0.00,0,0.0,0.00,0 days 00:15:00
7922,1679661000000,2023-03-24 12:30:00,77.68,77.68,77.68,77.68,0.0,0.00,0,0.0,0.00,0 days 00:15:00
7923,1679666400000,2023-03-24 14:00:00,77.69,77.80,77.69,77.77,17045.0,1325647.40,157,11805.0,918196.09,0 days 01:30:00


На самос деле отсутствие данных было на более продолжительном промежутке - удалим строки с отсутствием движения.

In [84]:
df.drop(index=[7918, 7919, 7920, 7921, 7922], axis=0, inplace=True)

In [85]:
df.diff_time.unique()

array([        'NaT',  900000000000, 5400000000000,             0],
      dtype='timedelta64[ns]')

In [86]:
df[df.diff_time == '0']

,Kline open time,time,Open price,High price,Low price,Close price,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,diff_time
7996,1679731200000,2023-03-25 08:00:00,77.96,77.96,77.76,77.91,29222.0,2274103.14,110,7332.0,571046.56,0 days


In [87]:
df.loc[7995:7996]

,Kline open time,time,Open price,High price,Low price,Close price,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,diff_time
7995,1679731200000,2023-03-25 08:00:00,77.96,77.96,77.76,77.91,29222.0,2274103.14,110,7332.0,571046.56,0 days 00:15:00
7996,1679731200000,2023-03-25 08:00:00,77.96,77.96,77.76,77.91,29222.0,2274103.14,110,7332.0,571046.56,0 days 00:00:00


удалим дубль

In [88]:
df.drop_duplicates(subset=['Kline open time'], inplace=True)

In [89]:
ti = pd.date_range(start = df.time.min(), end = df.time.max(), freq="15min") # интерполяционная сетка
ts = pd.DataFrame({'date':list(ti)})
ts['unixtime'] = (ts.date - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

In [90]:
upsampled_Open_price = np.interp(ts.unixtime, df['Kline open time'], df['Open price'])
upsampled_High_price = np.interp(ts.unixtime, df['Kline open time'], df['High price'])
upsampled_Low_price = np.interp(ts.unixtime, df['Kline open time'], df['Low price'])
upsampled_Close_price = np.interp(ts.unixtime, df['Kline open time'], df['Close price'])

In [91]:
data = pd.DataFrame({'unixtime': ts.unixtime, 'Open_price': upsampled_Open_price,
                       'High_price': upsampled_High_price, 'Low_price': upsampled_Low_price, 'Close_price': upsampled_Close_price})

In [92]:
data.insert(1, "time", pd.to_datetime(data.unixtime, unit='ms').to_list())
data.drop('unixtime', axis=1, inplace=True)

Поскольку прогноз нужно построить на час вперед, то добавим признаки из часового интервала.                                
Из аналитического трейдинга возьмем такие понятия как красные и зеленые свечи, и определим трети открытия и закрытия относительно ценового диапазона выбранного периода.                       
                                          
Мы добавим признаки по текущему промежутку на следующий временной промежуток - так мы имитируем ситуацию, когда трейдер видит свечу прошлого часа.                                           

In [93]:
# Фкц для определения трети 
def third_calc(High_price, Low_price, calc_price):
    movement = (High_price-Low_price)/3
    body = calc_price-Low_price
    if body<movement:
        third = 3
    elif body<2*movement:
        third = 2
    else:
        third = 1
    return third

In [94]:
# определяем, как аггрегировать данные в колонках
conversion = {
    'Open_price'  :  'first',  
    'High_price' :  'max', 
    'Low_price':  'min', 
    'Close_price': 'last'
}

data.set_index('time', inplace=True)
# уменьшаем частоту данных до 1 часа, аггрегируем данные, как описано выше, плюс смещаем на один шаг(час)
downsampled = data.resample('1H').apply(conversion).shift(1)
# возвращаем время из индекса в отдельную колонку
data = data.reset_index()
downsampled = downsampled.reset_index()

In [95]:
data = data.merge(downsampled, how = 'left', on='time', suffixes=('', '_by_hour')).fillna(method="ffill")

In [96]:
# Признак роста на часовом интервале (красная или зеленая свеча)
data['hour_is_green'] = np.where(data['Open_price_by_hour']<data['Close_price_by_hour'], 1,0)
# Признак трети открытия на часовом интервале
data['hour_open_thirds'] = data.apply(lambda x: third_calc(x.High_price_by_hour, x.Low_price_by_hour, x.Open_price_by_hour), axis=1)
# Признак трети закрытия на часовом интервале
data['hour_close_thirds'] = data.apply(lambda x: third_calc(x.High_price_by_hour, x.Low_price_by_hour, x.Close_price_by_hour), axis=1)

In [99]:
data.drop(['time','Open_price_by_hour','High_price_by_hour','Low_price_by_hour','Close_price_by_hour'], axis=1, inplace=True)
data.reset_index(inplace=True)

In [112]:
data.head(5)

,index,Open_price,High_price,Low_price,Close_price,hour_is_green,hour_open_thirds,hour_close_thirds
0,0,72.38,72.39,72.36,72.39,0,1,1
1,1,72.39,72.39,72.37,72.37,0,1,1
2,2,72.39,72.39,72.38,72.39,0,1,1
3,3,72.39,72.39,72.38,72.38,0,1,1
4,4,72.38,72.39,72.38,72.38,0,2,2


### train_val_test split с помощью numpy

In [113]:
# На тест мы оставим час и 15 минут, а трейн и валидацию поделим в соотношении 3 к 1
test = data.loc[10712:]
train, val = np.split(data.loc[:10712], [int(.75 *len(data))])

In [114]:
train.shape, val.shape, test.shape

((8037, 8), (2676, 8), (5, 8))

In [156]:
x_train = np.array(train[['index', 'hour_is_green','hour_open_thirds','hour_close_thirds']])#.reshape([-1,1])
y_train = np.array(train.Close_price)
x_val = np.array(val[['index', 'hour_is_green','hour_open_thirds','hour_close_thirds']])#.reshape([-1,1])
y_val = np.array(val.Close_price)
x_test = np.array(test[['index', 'hour_is_green','hour_open_thirds','hour_close_thirds']])#.reshape([-1,1])
y_test = np.array(test.Close_price)

### Normalize data

In [157]:
x_train_mean = [x_train[:,1].mean(), x_train[:,2].mean(), x_train[:,3].mean()] 
x_train_std = [x_train[:,1].std(), x_train[:,2].std(), x_train[:,3].std()] 

x_train[:, 1:] = (x_train[:,1:] - x_train_mean) / x_train_std
x_val[:, 1:] = (x_val[:,1:] - x_train_mean) / x_train_std
x_test[:, 1:] = (x_test[:,1:] - x_train_mean) / x_train_std

In [158]:
train_mean = y_train.mean()
train_std = y_train.std()

y_train_norm = (y_train - train_mean) / train_std
y_val_norm = (y_val - train_mean) / train_std
y_test_norm = (y_test - train_mean) / train_std

### LSTM

In [144]:
x_train.shape[0]

8037

In [145]:
model = Sequential()

In [149]:
model.add(L.LSTM(128, input_dim=x_train.shape[0]))

ValueError: Input 0 of layer lstm is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 128]

In [141]:
model = Sequential()
model.add(L.LSTM(128, return_sequences=True, input_shape=[32, x_train.shape[0], 1]))
model.add(L.LSTM(128, return_sequences=False))
model.add(L.Dense(1))
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, batch_size = 1, epochs=30)

ValueError: Input 0 of layer lstm_9 is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: [None, 32, 57888, 1]

In [138]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=x_train.shape[0], activation="relu"))
model_mlp.add(L.Dropout(0.7))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.7))